# Assembly steps

- Conda environment name: Assembly

- primarly for the illumina runs, paired-end sequences
- Steps are: 
    - QC with trimmomatic
        - bash script: Trimmomatic_QC.sh
    - PHIX filtering with bbmap
    - error correction with tedpole (from bbmap package)
    - assembly with metaSPAdes

In [ ]:
module load gcc12-env/12.3.0
module load miniconda3/24.11.1

cd $WORK/DATA
mkdir QCed_DATA PHIX_FILTERED ERROR_CORRECTED ASSEMBLIES

In [ ]:
TrimmomaticPE: Completed successfully

### DEBUGGING
#### Warning in slurm output file after running trimmomatic:
Exception in thread "Thread-0" java.lang.RuntimeException: Sequence and quality length don't match: 'AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC' vs 'F:FFFFF,FFF,F:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFF::F:FFFFFFF:FFFFFFFFFFFFF:FFF,FFFFFFFFFFFFFFFFFFFFFF,FFF:F:F,FFFF,:F:FFFFF,FFF::FF,,F,:FF:,,FF:FF,:F:FFF,F,:F,FF,:FFFFFFFFF:F:,FFFF,,FFF,:,,,F,,:F:F:::,,FF,:F,FF,:F:FFF:FFFFFF:FFFFFFFFFFFFFF'
	at org.usadellab.trimmomatic.fastq.FastqRecord.<init>(FastqRecord.java:25)
	at org.usadellab.trimmomatic.fastq.FastqParser.parseOne(FastqParser.java:89)
	at org.usadellab.trimmomatic.fastq.FastqParser.next(FastqParser.java:179)
	at org.usadellab.trimmomatic.threading.ParserWorker.run(ParserWorker.java:42)
	at java.base/java.lang.Thread.run(Thread.java:1570)
Exception in thread "Thread-1" java.lang.RuntimeException: Sequence and quality length don't match: 'GGGGGGGGGGGGGGGGGGGGGTGGGGTAGGGTTTGTATAGTGGTTGGGGTGGGGGGAGTATTGGTTGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGTCACACATAAGCAACATTTCGAAGGGTATGGTGATTTGGAATGACAAGAGTGTAAGATCGTAAGAGCGTCGTGAAGGGAAAGA' vs ',:FF:F,:FF,FFFFF:FFFFF,:FFFF,FF,,FF::,,:F:,:FF::,FFFF,F:FF:,F:FFFFFFF,F,FFFFF,,FFF:FF,::FF,FF,:F:F:F,FFFF,F,FFFFFFF,::F,FFFFF,F,,FFF:F,:,FFFF,F,,,FFFFF'
	at org.usadellab.trimmomatic.fastq.FastqRecord.<init>(FastqRecord.java:25)
	at org.usadellab.trimmomatic.fastq.FastqParser.parseOne(FastqParser.java:89)
	at org.usadellab.trimmomatic.fastq.FastqParser.next(FastqParser.java:179)
	at org.usadellab.trimmomatic.threading.ParserWorker.run(ParserWorker.java:42)
	at java.base/java.lang.Thread.run(Thread.java:1570)

##### Debugging
bioawk -cfastx 'length($seq) != length($qual)' RAWDATA/SRR23378605_1.fq.gz |wc
- output: 0    0    0
- The lengths are not different, so look into the file 


(FastqDump) [smomw681@nesh-login3 DATA]$ zgrep -A 3 "AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC" RAWDATA/SRR23378605_1.fastq.gz
AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC
+SRR23378605.25279883 HWI-A00245_BSF_0734:2:2119:9064:32503 length=151
F:FFFFF,FFF,F:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFF::F:FFFFFFF:FFFFFFFFFFFFF:FFF,FFFFFFFFFFFFFFFFFFFFFF,FFF:F:F,FFFF,:F:

(FastqDump) [smomw681@nesh-login3 DATA]$ zgrep -A 3 "GGGGGGGGGGGGGGGGGGGGGTGGGGTAGGGTTTGTATAGTGGTTGGGGTGGGGGGAGTATTGGTTGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGTCACACATAAGCAACATTTCGAAGGGTATGGTGATTTGGAATGACAAGAGTGTAAGATCGTAAGAGCGTCGTGAAGGGAAAGA" RAWDATA/SRR23378605_1.fastq.gz
gzip: RAWDATA/SRR23378605_1.fastq.gz: invalid compressed data--format violated
